<a href="https://colab.research.google.com/github/ToqaBahaa/CellulaTasks/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import zipfile

drive.mount('/content/drive')

zip_path = '/content/drive/MyDrive/TeethDataSet.zip'
extract_path = '/content/drive/MyDrive/TeethDataSet'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

!ls /content/drive/MyDrive/TeethDataSet

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Teeth_Dataset


In [2]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dir = '/content/drive/MyDrive/TeethDataSet/Teeth_Dataset/Training'
val_dir = '/content/drive/MyDrive/TeethDataSet/Teeth_Dataset/Validation'
test_dir = '/content/drive/MyDrive/TeethDataSet/Teeth_Dataset/Testing'

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [3]:
import torch
import torch.nn as nn
import torchvision.models as models

model = models.resnet18(pretrained=True)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)

model = torch.compile(model)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 188MB/s]


In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    print(f'Validation Loss: {val_loss/len(val_loader):.4f}')


Epoch [1/10], Loss: 1.0099
Validation Loss: 1.1389


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/TeethClassification/saved_model.pth')


In [ ]:
import streamlit as st
from PIL import Image
import torch
import torchvision.transforms as transforms

model.load_state_dict(torch.load('/content/drive/MyDrive/TeethClassification/saved_model.pth'))
model.eval()


image_path = '/content/drive/MyDrive/TeethDataSet/Teeth_Dataset/Validation/CaS/a_80_0_1196.jpg'
image = Image.open(image_path)
st.image(image, caption='Loaded Image.', use_column_width=True)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

image = transform(image).unsqueeze(0)

with torch.no_grad():
    output = model(image)
    _, predicted = torch.max(output, 1)

class_names = ['CaS', 'CoS', 'Gum', 'MC', 'OC', 'OLP', 'OT']
st.write(f'Predicted Class: {class_names[predicted]}')
